In [581]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import warnings
import json
warnings.filterwarnings("ignore")
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

def forward_fill_non_zero(df, column_name):
    last_non_zero = None
    for idx in df.index:
        value = df.loc[idx, column_name]
        if value != 0:
            last_non_zero = value
        elif last_non_zero is not None:
            df.at[idx, column_name] = last_non_zero
    return df

def calc_stats(df):

    stats_df = pd.DataFrame()
    stats_df['mean'] = df.mean()
    stats_df['std'] = df.std()
    # print('calculated')
    return stats_df

In [695]:
from datetime import datetime, timezone, timedelta

### profitlosscontrol

In [691]:
from_date=datetime(2023,12, 16)
to_date=datetime.now()
order_hist = mt5.history_deals_get(from_date, to_date)

In [692]:
hist_dict = {}
for pos in order_hist:
    if pos.profit != 0.0:
        hist_dict[pos.ticket] = pos

In [737]:
new_lst = []

for key, pos in hist_dict.items():
    new_lst.append([pos.time, pos.swap, pos.commission, pos.profit])

In [740]:
for pos in new_lst:
    unix_time = pos[0]
    date_time = datetime.fromtimestamp(unix_time)
    pos[0] = date_time.strftime('%Y-%m-%d %H:%M:%S')

In [767]:
start_time = '00:00:00'

In [774]:
hist_df

,Swap,Commission,Profit
Time,,,
2023-12-20 18:32:00,5.80,-3.50,-819.67
2023-12-20 18:32:00,-16.49,-3.50,461.36
2023-12-20 18:32:01,-53.72,-3.50,812.63
2023-12-20 18:32:01,-1.65,-3.50,658.71
2023-12-20 18:32:02,38.89,-3.50,1941.65
2023-12-20 18:32:02,-6.16,-3.50,915.67
2023-12-20 18:32:03,-27.88,-3.50,-420.91
2023-12-22 15:12:18,-8.29,-5.71,693.73
2023-12-22 15:12:19,-3.30,-10.89,-2455.53


In [773]:
hist_df = pd.DataFrame(new_lst)
hist_df.columns = 'Time', 'Swap', 'Commission', 'Profit'
hist_df = hist_df.set_index('Time')
hist_df.loc[start_time]

KeyError: '00:00:00'

In [776]:
mt5.positions_get()

()

In [953]:
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'Regres Close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)
    
def close_all():
    close_positions = []
    open_positions = mt5.positions_get()
    open_positions
    for i in open_positions:
        close_positions.append(i)
        
    for pos in close_positions:
        close_position(pos)
        
class ProfitLossControl:
    
    def __init__(self, max_daily_dd, profit_threshold, hedging_threshold, starting_equity):
        self.max_daily_dd = max_daily_dd
        self.profit_threshold = profit_threshold
        self.hedging_threshold = hedging_threshold
        self.starting_equity = starting_equity
    
    def start_equity(self):

        gmt_plus_2 = pytz.timezone('EET')  # Eastern European Time is typically GMT+2

        # Get the current time in UTC and convert it to GMT+2
        now_utc = datetime.now(timezone.utc)
        time = now_utc.astimezone(gmt_plus_2)
        now_gmt_plus_2 = now_utc.astimezone(gmt_plus_2)

        # Create a datetime object for the reset time (00:00:01) on the same day in GMT+2
        reset_time_today = now_gmt_plus_2.replace(hour=0, minute=0)

        if now_gmt_plus_2 != reset_time_today:
            info = mt5.account_info()
            start_equity = info.equity
            print(f"Starting equity is {start_equity}")
            return start_equity

        else:
            print(f"{countdown} remaining until equity reset")
            return countdown 

    def day_pnl(self, start_equity): # Return current days equity drawdow / profit as a percentage from the start of the day. 
        
        # Define the GMT+2 timezone
        gmt_plus_2 = pytz.timezone('EET')  # Eastern European Time is typically GMT+2

        # Get the current time in UTC and convert it to GMT+2
        now_utc = datetime.now(timezone.utc)
        time = now_utc.astimezone(gmt_plus_2)
        now_gmt_plus_2 = now_utc.astimezone(gmt_plus_2)

        # Create a datetime object for the reset time (00:00:01) on the same day in GMT+2
        reset_time_today = now_gmt_plus_2.replace(hour=0, minute=0, second=1, microsecond=0)

        # Calculate the countdown to the reset time
        if now_gmt_plus_2 >= reset_time_today:
            # If the current time is after the reset time, calculate the countdown to the next day's reset time
            reset_time_tomorrow = reset_time_today + timedelta(days=1)
            countdown = reset_time_tomorrow - now_gmt_plus_2
        else:
            # If the current time is before today's reset time, calculate the countdown to today's reset time
            countdown = reset_time_today - now_gmt_plus_2
        
        account_info = mt5.account_info()
        equity = account_info.balance + account_info.profit
        
        day_pnl = equity - start_equity

        print(day_pnl)
        print(f"Time until next reset: {countdown}")
        
        return int(day_pnl)
        
    def daily_dd_control(self, day_pnl): 
        
        if day_pnl <= (self.max_daily_dd / 1.35):
            print('Closing all positions for the day')
            close_all()
        else:
            print(f'Threshold not reached. Current day pnl percent: {day_pnl}')
        
obj = ProfitLossControl(-0.03, 0.03, 0.00125, 190000)

In [954]:
t1ab = obj.start_equity()

Starting equity is 5901.43


In [942]:
countdown

datetime.timedelta(seconds=42558, microseconds=391449)

In [955]:
day_equity = obj.day_pnl(t1ab)

-0.5300000000006548
Time until next reset: 8:23:57.509222


In [945]:
obj.day_pnl(t1ab)

7.819999999999709
Time until next reset: 8:28:09.460358


7

In [934]:
day_equity = -0.03

In [935]:
obj.daily_dd_control(-0.03)

Closing all positions for the day


In [951]:

def get_countdown():
    gmt_plus_2 = pytz.timezone('EET')  # Eastern European Time is typically GMT+2

    # Get the current time in UTC and convert it to GMT+2
    now_utc = datetime.now(timezone.utc)
    time = now_utc.astimezone(gmt_plus_2)
    now_gmt_plus_2 = now_utc.astimezone(gmt_plus_2)

    # Create a datetime object for the reset time (00:00:01) on the same day in GMT+2
    reset_time_today = now_gmt_plus_2.replace(hour=0, minute=0)

    if now_gmt_plus_2 == reset_time_today:
        info = mt5.account_info()
        start_equity = info.equity
        print(f"Starting equity is {start_equity}")
        return start_equity

    else:
        print(f"{countdown} remaining until equity reset")
        return countdown

In [952]:
get_countdown()

11:49:18.391449 remaining until equity reset


datetime.timedelta(seconds=42558, microseconds=391449)

## Data Gathering

In [936]:
combined_dfs_m = {}
for country in count_codes:

    # Define the endpoint parameters
    flow = 'BIS,WS_EER_M,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
    key = f'M.N.N.{country[1]}'
    start_period = '2000'  # Example: Start year 2000
    end_period = '2025'  # Example: End year 2020
    detail = 'full'  # Example: All data and documentation

    # Construct the endpoint URL
    endpoint_url = f'{base_url}/data/{flow}/{key}/all'

    # Define the query parameters
    query_params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': detail
    }

    # Make the GET request
    response = requests.get(endpoint_url, params=query_params)

    # Check for a successful response
    if response.status_code == 200:
        # Assign the text of the response to xml_data
        xml_data = response.text

        # Parse the XML data
        root = ET.fromstring(xml_data)

        # Initialize empty lists to store the data
        time_periods = []
        obs_values = []

        # Iterate through the XML and extract the desired information
        for obs in root.findall(".//Obs"):
            time_period = obs.get('TIME_PERIOD')
            obs_value = obs.get('OBS_VALUE')
            time_periods.append(time_period)
            obs_values.append(obs_value)

        # Create a DataFrame
        df = pd.DataFrame({
            'Time_Period': time_periods,
            'OBS_Value': obs_values
        })

        df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
        # Drop rows with NaN values
        df.dropna(subset=['OBS_Value'], inplace=True)
        df['OBS_Value'] = df['OBS_Value'].astype(float)
        df['Time_Period'] = pd.to_datetime(df['Time_Period'])
        
        df = df.set_index('Time_Period')
        print(f"Getting {country[0]}'s rates")
        rates = get_rates(country[0], mt5.TIMEFRAME_D1, 2500)
        
        combined = pd.concat([df[-len(rates):], rates['close']], join = 'outer', axis = 1)
        
        combined_dfs_m[country[1]] = combined.dropna()

    else:
        print(f'Failed to retrieve data: {response.status_code}')

Getting AUDUSD.a's rates
Getting USDCAD.a's rates
Getting USDCHF.a's rates
Getting GBPUSD.a's rates
Getting USDJPY.a's rates
Getting NZDUSD.a's rates
Getting EURUSD.a's rates


In [937]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

combined_dfs = {}

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

count_codes = [['AUDUSD.a','AU'], ['USDCAD.a', 'CA'],
               ['USDCHF.a','CH'], ['GBPUSD.a', 'GB'],
               ['USDJPY.a', 'JP'], ['NZDUSD.a', 'NZ'], 
               ['EURUSD.a', 'XM']]

for country in count_codes:

    # Define the endpoint parameters
    flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
    key = f'D.N.N.{country[1]}'
    start_period = '2000'  # Example: Start year 2000
    end_period = '2025'  # Example: End year 2020
    detail = 'full'  # Example: All data and documentation

    # Construct the endpoint URL
    endpoint_url = f'{base_url}/data/{flow}/{key}/all'

    # Define the query parameters
    query_params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': detail
    }

    # Make the GET request
    response = requests.get(endpoint_url, params=query_params)

    # Check for a successful response
    if response.status_code == 200:
        # Assign the text of the response to xml_data
        xml_data = response.text

        # Parse the XML data
        root = ET.fromstring(xml_data)

        # Initialize empty lists to store the data
        time_periods = []
        obs_values = []

        # Iterate through the XML and extract the desired information
        for obs in root.findall(".//Obs"):
            time_period = obs.get('TIME_PERIOD')
            obs_value = obs.get('OBS_VALUE')
            time_periods.append(time_period)
            obs_values.append(obs_value)

        # Create a DataFrame
        df = pd.DataFrame({
            'Time_Period': time_periods,
            'OBS_Value': obs_values
        })

        df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
        # Drop rows with NaN values
        df.dropna(subset=['OBS_Value'], inplace=True)
        df['OBS_Value'] = df['OBS_Value'].astype(float)
        df['Time_Period'] = pd.to_datetime(df['Time_Period'])
        
        df = df.set_index('Time_Period')
        print(f"Getting {country[0]}'s rates")
        rates = get_rates(country[0], mt5.TIMEFRAME_D1, 2500)
        
        combined = pd.concat([df[-len(rates):], rates['close']], join = 'outer', axis = 1)
        
        combined_dfs[country[1]] = combined.dropna()

    else:
        print(f'Failed to retrieve data: {response.status_code}')

Getting AUDUSD.a's rates
Getting USDCAD.a's rates
Getting USDCHF.a's rates
Getting GBPUSD.a's rates
Getting USDJPY.a's rates
Getting NZDUSD.a's rates
Getting EURUSD.a's rates


In [938]:
# Compute Expected Returns
weekly_dfs = {}
features = ['OBS_Value', 'close']

for name, dfs in combined_dfs.items():
    df = combined_dfs[name]
    # Assuming 'df' is your DataFrame
    df['date'] = pd.to_datetime(df.index)
    df.set_index('date', inplace=True)

    # Resample to get the last value of each week
    weekly = df.resample('W').last()
    weekly_dfs[name] = weekly
    
    # Add a new column for Monthly OBS Values in the weekly DataFrame
    weekly_dfs[name]['Monthly_OBS_Value'] = np.nan

    # Iterate through the monthly DataFrame
    for date, row in combined_dfs_m[name].iterrows():
        # Find all weekly dates that are in the same month and year as the monthly date
        mask = (weekly_dfs[name].index.month == date.month) & (weekly_dfs[name].index.year == date.year)
        weekly_dfs[name].loc[mask, 'Monthly_OBS_Value'] = row['OBS_Value']

    # Forward fill the NaN values in the Monthly_OBS_Value column
    weekly_dfs[name]['Monthly_OBS_Value'] = weekly_dfs[name]['Monthly_OBS_Value'].ffill()
    
for df in weekly_dfs.values():
    df['W_EER_ret'] = df['OBS_Value'].pct_change()
    df['M_EER_ret'] = df['Monthly_OBS_Value'].pct_change()
    df['close_ret'] = df['close'].pct_change()
    df['ret_diff'] = df['close_ret'] - df['W_EER_ret']

In [939]:
weekly_dfs['AU']

,OBS_Value,close,Monthly_OBS_Value,W_EER_ret,M_EER_ret,close_ret,ret_diff
date,,,,,,,
2014-06-08,121.06,0.93325,NaN,NaN,NaN,NaN,NaN
2014-06-15,121.81,0.93998,NaN,0.006195,NaN,0.007211,0.001016
2014-06-22,121.76,0.93867,NaN,-0.000410,NaN,-0.001394,-0.000983
2014-06-29,121.67,0.94239,NaN,-0.000739,NaN,0.003963,0.004702
2014-07-06,120.95,0.93628,NaN,-0.005918,NaN,-0.006484,-0.000566
...,...,...,...,...,...,...,...
2023-12-10,102.83,0.65758,102.04,-0.001844,0.0,-0.014861,-0.013017
2023-12-17,103.38,0.67018,102.04,0.005349,0.0,0.019161,0.013813
2023-12-24,104.56,0.67972,102.04,0.011414,0.0,0.014235,0.002821


In [138]:
last_update = {}

for name, df in weekly_dfs.items():
    last_update[name] = df['W_EER_ret'].iloc[-1]

In [139]:
last_update.keys()
symbols = ['AUDUSD.a', 'USDCAD.a', 'USDCHF.a', 'GBPUSD.a', 'USDJPY.a', 'NZDUSD.a', 'EURUSD.a']

last_update = {symbols[i]: value for i, (key, value) in enumerate(last_update.items())}

In [579]:
last_update

{'AUDUSD.a': 0.005348633667217717,
 'USDCAD.a': 0.010298049311813218,
 'USDCHF.a': -0.00220731061274948,
 'GBPUSD.a': 0.0011580775911985697,
 'USDJPY.a': 0.00992167101827679,
 'NZDUSD.a': -0.0012782694198624212,
 'EURUSD.a': 0.0059632100262785315}

## Order Sending / Closing Logic

In [382]:
def close_all():
    close_positions = []
    open_positions = mt5.positions_get()
    open_positions
    for i in open_positions:
        close_positions.append(i)
        
    for pos in close_positions:
        close_position(pos)
        
def close_position(position, comment):
    
    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": comment,
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)

In [35]:
def send_order(symbol, side, lot, comment):
    
    if side.lower() == 'sell':
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    elif side.lower() == 'buy':
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 5,
        "magic": 234000,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    result

In [383]:
all_positions = mt5.positions_get()
for position in all_positions:
    print(position)

TradePosition(ticket=548724611, time=1703057580, time_msc=1703057580167, time_update=1703057580, time_update_msc=1703057580167, type=0, magic=234000, identifier=548724611, reason=3, volume=1.0, price_open=0.67699, sl=0.0, tp=0.0, price_current=0.67972, swap=-25.6, profit=401.35, symbol='AUDUSD.a', comment='EER', external_id='')
TradePosition(ticket=548724617, time=1703057580, time_msc=1703057580645, time_update=1703057580, time_update_msc=1703057580645, type=1, magic=234000, identifier=548724617, reason=3, volume=1.0, price_open=1.33499, sl=0.0, tp=0.0, price_current=1.32675, swap=-27.42, profit=913.26, symbol='USDCAD.a', comment='EER', external_id='')
TradePosition(ticket=548724638, time=1703057581, time_msc=1703057581125, time_update=1703057581, time_update_msc=1703057581125, type=0, magic=234000, identifier=548724638, reason=3, volume=1.0, price_open=0.86149, sl=0.0, tp=0.0, price_current=0.85574, swap=54.44, profit=-988.79, symbol='USDCHF.a', comment='EER', external_id='')
TradePos

# Risk Management

**High level approach**

ARIMA model to forecast EER values (from week-1) to be used as a feature in a regression model along with the EER vals for week0 along with the EER value difference vs EER value forecasted

Obtain an expected return (range?) to then create an MVO portfolio to run in a monte carlo simulation to obtain the most optimal weightings/allocation (optimised for sharpe ratio)

Steps
1. Create ARIMA and EER Val forecasts 
    - Run statistical analysis to incorporate differing variations 
2. Train a regression model on data to obtain expected returns
3. 

### ARIMA value

Forecast next EER value for each currency for the next week. Use forecasted EER values AND the difference between forecasted EER values vs actual as features for linear regression model.

In [384]:
import itertools
import warnings
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

def evaluate_arima_model_aic_bic(series, arima_order):
    '''Evaluates ARIMA model for AIC and BIC'''
    try:
        model = ARIMA(series, order=arima_order)
        model_fit = model.fit()
        aic = model_fit.aic
        bic = model_fit.bic
        return aic, bic
    except Exception as e:
        print(f'Error with ARIMA order {arima_order}: {e}')
        return float('inf'), float('inf')

def find_best_arima_model(series, p_values, d_values, q_values):
    '''Find the best ARIMA model based on AIC and BIC'''
    best_aic, best_bic, best_cfg = float("inf"), float("inf"), None
    for p, d, q in itertools.product(p_values, d_values, q_values):
        order = (p, d, q)
        aic, bic = evaluate_arima_model_aic_bic(series, order)
        if aic < best_aic or bic < best_bic:
            best_aic, best_bic, best_cfg = aic, bic, order
            print(f'ARIMA{order} AIC: {aic}, BIC: {bic}')
    print(f'Best ARIMA{best_cfg} AIC: {best_aic}, BIC: {best_bic}')
    return best_cfg

In [400]:
weekly_dfs['CA']

,OBS_Value,close,Monthly_OBS_Value,W_EER_ret,M_EER_ret,close_ret,ret_diff
date,,,,,,,
2014-05-25,118.63,1.08603,NaN,NaN,NaN,NaN,NaN
2014-06-01,119.35,1.08438,NaN,0.006069,NaN,-0.001519,-0.007589
2014-06-08,118.37,1.09316,NaN,-0.008211,NaN,0.008097,0.016308
2014-06-15,119.26,1.08551,NaN,0.007519,NaN,-0.006998,-0.014517
2014-06-22,119.58,1.07576,NaN,0.002683,NaN,-0.008982,-0.011665
...,...,...,...,...,...,...,...
2023-11-19,100.03,1.37184,100.13,0.002706,0.0,-0.005870,-0.008576
2023-11-26,100.19,1.36336,100.13,0.001600,0.0,-0.006181,-0.007781
2023-12-03,101.12,1.34972,100.13,0.009282,0.0,-0.010005,-0.019287


In [409]:
arma_forecasts = {}

series = weekly_dfs['AU']['OBS_Value']
best_arima_order = (2, 1, 2)

# Store the forecasts for each DataFrame
forecasts = []

# Iterate over the series, retraining the model on the most recent 100 data points and forecasting one step ahead
for end in range(100, len(series)):
    if end % 50 == 0:
        print(f'{end + 50} data points ran')
    train_data = series.iloc[end-100:end]
    model = ARIMA(train_data, order=best_arima_order)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecasts.append(forecast[0])

arma_forecasts['AU'] = pd.DataFrame(forecasts, columns=['Forecast'])

150 data points ran
200 data points ran
250 data points ran
300 data points ran
350 data points ran
400 data points ran
450 data points ran
500 data points ran


In [410]:
nans = pd.DataFrame({'Forecast': [np.nan] * 100})

In [411]:
au_eer_forecasts = pd.DataFrame(arma_forecasts['AU'])

In [502]:
aligned_df = pd.concat([nans, au_eer_forecasts]).reset_index(drop=True)
t1 = weekly_dfs['AU'].reset_index(drop = True)
t1a = pd.concat([t1, aligned_df], axis = 1).dropna()
t1a['forecast_change'] = t1a['Forecast'].pct_change()
t1a['real_vs_forecast'] = t1a['OBS_Value'] - t1a['Forecast']
t1a['r_v_f_ratio'] = t1a['OBS_Value'] / t1a['Forecast']
t1a

,OBS_Value,close,Monthly_OBS_Value,W_EER_ret,M_EER_ret,close_ret,ret_diff,Forecast,forecast_change,real_vs_forecast,r_v_f_ratio
100,111.70,0.77073,110.40,0.005129,0.0,-0.001697,-0.006826,111.214791,NaN,0.485209,1.004363
101,109.04,0.76055,110.40,-0.023814,0.0,-0.013208,0.010606,111.473987,0.002331,-2.433987,0.978165
102,105.77,0.73638,110.40,-0.029989,0.0,-0.031780,-0.001791,108.946741,-0.022671,-3.176741,0.970841
103,105.15,0.72697,110.40,-0.005862,0.0,-0.012779,-0.006917,106.230829,-0.024929,-1.080829,0.989826
104,105.10,0.72208,110.40,-0.000476,0.0,-0.006727,-0.006251,105.973673,-0.002421,-0.873673,0.991756
...,...,...,...,...,...,...,...,...,...,...,...
495,101.76,0.65142,102.04,0.010125,0.0,0.024213,0.014088,101.201704,-0.009347,0.558296,1.005517
496,102.54,0.65801,102.04,0.007665,0.0,0.010116,0.002451,101.155174,-0.000460,1.384826,1.013690
497,103.02,0.66750,102.04,0.004681,0.0,0.014422,0.009741,103.072937,0.018959,-0.052937,0.999486
498,102.83,0.65758,102.04,-0.001844,0.0,-0.014861,-0.013017,102.985700,-0.000846,-0.155700,0.998488


In [460]:
t1a['close_ret'].corr(t1a['real_vs_forecast_change'])

-0.02800769130933722

In [431]:
variables = [['W_EER_ret', 'OBS_Value', 'Forecast', 'real_vs_forecast']]

In [438]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [518]:
X = t1a[['r_v_f_ratio', 'W_EER_ret']]
y = t1a['close_ret']

X = scaler.fit_transform(X) 

In [521]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# Create a Decision Tree Regressor model
model = DecisionTreeRegressor()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Calculate the performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Coefficient of Determination (R^2): {r2}")

Mean Squared Error (MSE): 0.00011891429932442882
Coefficient of Determination (R^2): 0.43961711835228123


In [514]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Creating the linear regression model
model = LinearRegression()

# Training the model
model.fit(X_train, y_train)

# Making predictions using the testing set
y_pred = model.predict(X_test)

# The coefficients
print('Coefficients:', model.coef_)
# The mean squared error
print('Mean squared error (MSE): %.6f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination (R^2): %.6f' % r2_score(y_test, y_pred))

Coefficients: [-0.00037561 -0.00193706]
Mean squared error (MSE): 0.000152
Coefficient of determination (R^2): -0.089668


In [399]:
arma_forecasts = {}

for df_name, df in weekly_dfs.items():
    print(f'Iterating through {df_name}')
    
    series = df['OBS_Value']
    best_arima_order = (2, 1, 2)

    # Store the forecasts for each DataFrame
    forecasts = []

    # Iterate over the series, retraining the model on the most recent 100 data points and forecasting one step ahead
    for end in range(100, len(series)):
        if end % 50 == 0:
            print(f'{end + 50} data points ran')
        train_data = series.iloc[end-100:end]
        model = ARIMA(train_data, order=best_arima_order)
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=1)
        forecasts.append(forecast[0])

    arma_forecasts[df_name] = pd.DataFrame(forecasts, columns=['Forecast'])


Iterating through AU
150 data points ran
200 data points ran
250 data points ran
300 data points ran
350 data points ran
400 data points ran
450 data points ran
500 data points ran
Iterating through CA
150 data points ran


LinAlgError: LU decomposition error.

In [ ]:
arma_forecast

In [394]:
arma_forecasts = {}
for df in weekly_dfs:
    print(f'Iterating through {df}')
    
    series = weekly_dfs[df]['OBS_Value']

    best_arima_order = (2, 1, 2)

    best_model = ARIMA(series, order=best_arima_order)
    best_model_fit = best_model.fit()
    forecast = best_model_fit.forecast(steps=100)
    arma_forecasts[df] = pd.DataFrame(forecast)

Iterating through AU
Iterating through CA
Iterating through CH
Iterating through GB
Iterating through JP
Iterating through NZ
Iterating through XM


In [140]:
forecast_diff = {}

for key, val in last_update.items():
    for key2, val2 in arma_forecasts.items():
        if key2 in key:
            forecast_diff[key2] = (val - val2) / val
        else:
            if key == 'EURUSD.a':
                if key2 == 'XM':
                    forecast_diff[key2] = (val - val2) / val

In [141]:
forecast_diff

{'AU': 2023-12-24    1.044799
 Freq: W-SUN, dtype: float64,
 'CA': 2023-12-24    1.025587
 Freq: W-SUN, dtype: float64,
 'CH': 2023-12-24    0.638441
 Freq: W-SUN, dtype: float64,
 'GB': 2023-12-24    1.130399
 Freq: W-SUN, dtype: float64,
 'JP': 2023-12-24    1.036089
 Freq: W-SUN, dtype: float64,
 'NZ': 2023-12-24    0.859181
 Freq: W-SUN, dtype: float64,
 'XM': 2023-12-24    1.017419
 Freq: W-SUN, dtype: float64}

### Regression Model / Monte Carlo Sim / weights

In [212]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [272]:
models = {}
best_model_vars_dict = {}

X_vars = [['W_EER_ret', 'M_EER_ret'], ['W_EER_ret']]
for df in weekly_dfs:
    best_score = float('-inf')  # Initialize the best score as negative infinity
    best_model = None  # Initialize the best model
    for variables in X_vars:
        print(f" --------- Testing variable list {variables} --------- ")
        print(f"Looping through {df}")
        X = weekly_dfs[df][variables].dropna()
        y = weekly_dfs[df]['close_ret'].dropna()
        y = y.iloc[-len(X):]

        # Splitting the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

        # Creating the linear regression model
        model = LinearRegression()
        
        # Training the model
        model.fit(X_train, y_train)

        # Making predictions using the testing set
        y_pred = model.predict(X_test)

        # The coefficients
        # print('Coefficients:', model.coef_)
        # The mean squared error
        # print('Mean squared error (MSE): %.6f' % mean_squared_error(y_test, y_pred))
        # The coefficient of determination: 1 is perfect prediction
        print('Coefficient of determination (R^2): %.6f' % r2_score(y_test, y_pred))
        new_score = r2_score(y_test, y_pred)
        # Check if this model is better
        if new_score > best_score:
            print(f"**Model with {variables} performs better. Updating best model.**")
            best_score = new_score
            best_model = model
            best_model_vars = variables
            
    models[df] = best_model
    best_model_vars_dict[df] = best_model_vars  # Store the best variables in the dictionary

    stats[df] = calc_stats(weekly_dfs[df][best_model_vars])

 --------- Testing variable list ['W_EER_ret', 'M_EER_ret'] --------- 
Looping through AU
Coefficient of determination (R^2): 0.507417
**Model with ['W_EER_ret', 'M_EER_ret'] performs better. Updating best model.**
 --------- Testing variable list ['W_EER_ret'] --------- 
Looping through AU
Coefficient of determination (R^2): 0.648780
**Model with ['W_EER_ret'] performs better. Updating best model.**
 --------- Testing variable list ['W_EER_ret', 'M_EER_ret'] --------- 
Looping through CA
Coefficient of determination (R^2): 0.586411
**Model with ['W_EER_ret', 'M_EER_ret'] performs better. Updating best model.**
 --------- Testing variable list ['W_EER_ret'] --------- 
Looping through CA
Coefficient of determination (R^2): 0.655828
**Model with ['W_EER_ret'] performs better. Updating best model.**
 --------- Testing variable list ['W_EER_ret', 'M_EER_ret'] --------- 
Looping through CH
Coefficient of determination (R^2): 0.310929
**Model with ['W_EER_ret', 'M_EER_ret'] performs better. 

In [238]:
all_returns = pd.DataFrame()
for sym in count_codes:
    # print(f'Iterating thru {sym}')
    rates = get_rates(sym[0], mt5.TIMEFRAME_D1, 2000)
    rates = rates['close']
    rates = rates.rename(f'{sym[0]}_close')
    all_returns = pd.concat([all_returns, rates], axis = 1)
latest_prices = all_returns.iloc[-1]

In [278]:
best_model_vars_dict

{'AU': ['W_EER_ret'],
 'CA': ['W_EER_ret'],
 'CH': ['W_EER_ret'],
 'GB': ['W_EER_ret'],
 'JP': ['W_EER_ret', 'M_EER_ret'],
 'NZ': ['W_EER_ret', 'M_EER_ret'],
 'XM': ['W_EER_ret']}

In [279]:
for i in range(20):

    if (i + 1) % 250 == 0:
        print(f'{i + 1} simulations ran')

    expected_return_lst = {}
    for df, values in stats.items():
        mean = values['mean']
        std = values['std']

        for df2, lr_model in models.items():
            if df == df2:
                num_features = len(best_model_vars_dict[df2])  # Number of features the model was trained on
                # Ensure mean and std have correct dimensions
                mean_values = np.full(num_features, mean)
                std_values = np.full(num_features, std)
                # Generate random values for each feature
                random_eer_ret = np.random.normal(mean_values, std_values, (1, num_features))
                # Predict using the model
                expected_return_lst[df2] = lr_model.predict(random_eer_ret)
            
    lst = []
    for i in expected_return_lst.values():
        lst.append(i)

In [316]:
import numpy as np
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices 

num_simulations = 10000
all_weights = []

for i in range(num_simulations):

    if (i + 1) % 250 == 0:
        print(f'{i + 1} simulations ran')

    expected_return_lst = {}
    for df, values in stats.items():
        mean = values['mean']
        std = values['std']

        for df2, lr_model in models.items():
            if df == df2:
                num_features = len(best_model_vars_dict[df2])  # Number of features the model was trained on
                # Ensure mean and std have correct dimensions
                mean_values = np.full(num_features, mean)
                std_values = np.full(num_features, std)
                # Generate random values for each feature
                random_eer_ret = np.random.normal(mean_values, std_values, (1, num_features))
                # Predict using the model
                expected_return_lst[df2] = lr_model.predict(random_eer_ret)
            
    lst = []
    for i in expected_return_lst.values():
        lst.append(i)
        
    highest_return = max(lst)
    desired_scale = 0.05  # 5%
    scaling_factor = desired_scale / highest_return
    
    # Apply the scaling factor
    scaled_returns = [ret * scaling_factor for ret in lst] 
    scaled_values = [x[0] * 100 for x in scaled_returns]
    fin = []
    for i in scaled_values:
        fin.append(abs(i))
    
    # Calculate expected returns and the covariance matrix
    mu = fin
    S = risk_models.sample_cov(all_returns)

    # Optimize for the minimised volatility
    ef = EfficientFrontier(mu, S, weight_bounds = (0.05, 0.25))
    weights = ef.efficient_risk(0.025, market_neutral = False)
    cleaned_weights = ef.clean_weights()
    
    exp_ret, exp_vol, sharpe = ef.portfolio_performance()
    da = DiscreteAllocation(
    cleaned_weights, latest_prices, 
    total_portfolio_value = 10000, short_ratio = None)
    
    # Perform the discrete allocation
    allocation, leftover = da.lp_portfolio()
    # Calculate the total investment in each asset
    total_investment = {ticker: allocation[ticker] * latest_prices[ticker] for ticker in allocation}

    # Total value of the portfolio
    total_portfolio_value = sum(total_investment.values()) + leftover

    # Convert to percentages
    portfolio_percentages = {ticker: (value / total_portfolio_value) * 100 for ticker, value in total_investment.items()}
    
    # Store the weights from each simulation
    # all_weights[i, :] = np.array(list(portfolio_percentages.values()))
    all_weights.append([portfolio_percentages, exp_ret, exp_vol])

250 simulations ran
500 simulations ran
750 simulations ran
1000 simulations ran
1250 simulations ran
1500 simulations ran
1750 simulations ran
2000 simulations ran
2250 simulations ran
2500 simulations ran
2750 simulations ran
3000 simulations ran
3250 simulations ran
3500 simulations ran
3750 simulations ran
4000 simulations ran
4250 simulations ran
4500 simulations ran
4750 simulations ran
5000 simulations ran
5250 simulations ran
5500 simulations ran
5750 simulations ran
6000 simulations ran
6250 simulations ran
6500 simulations ran
6750 simulations ran
7000 simulations ran
7250 simulations ran
7500 simulations ran
7750 simulations ran
8000 simulations ran
8250 simulations ran
8500 simulations ran
8750 simulations ran
9000 simulations ran
9250 simulations ran
9500 simulations ran
9750 simulations ran
10000 simulations ran


In [317]:
weights_list = []
returns_list = []
volatility_list = []

# Extract values into separate lists
for item in all_weights:
    weights_list.append(item[0])  # Portfolio weights dictionary
    returns_list.append(item[1])  # Expected return
    volatility_list.append(item[2])  # Expected volatility

# Convert weights to DataFrame for easier mean calculation
df_weights = pd.DataFrame(weights_list)

# Calculate mean weights, returns, and volatility
mean_weights = df_weights.mean()
mean_return = np.mean(returns_list)
mean_volatility = np.mean(volatility_list)

print("Mean Portfolio Weights:")
print(mean_weights)
print("\nMean Expected Return:", mean_return)
print("Mean Expected Volatility:", mean_volatility)

Mean Portfolio Weights:
USDCAD.a_close    22.780229
USDCHF.a_close    20.829847
GBPUSD.a_close     9.302940
USDJPY.a_close    13.285449
NZDUSD.a_close    13.377707
EURUSD.a_close    14.508887
AUDUSD.a_close    11.256490
dtype: float64

Mean Expected Return: 6.648499074651092
Mean Expected Volatility: 0.0248788400838889


In [318]:
import pandas as pd
import numpy as np

weights_list = []
returns_list = []
volatility_list = []

# Extract values into separate lists
for item in all_weights:
    weights_list.append(item[0])  # Portfolio weights dictionary
    returns_list.append(item[1])  # Expected return
    volatility_list.append(item[2])  # Expected volatility

# Convert weights to DataFrame for easier mean and IQR calculation
df_weights = pd.DataFrame(weights_list)

# Calculate mean weights and IQR for each asset
mean_weights = df_weights.mean()
iqr_weights = df_weights.quantile(0.75) - df_weights.quantile(0.25)
iqr_25_weights = df_weights.quantile(0.25)
iqr_75_weights = df_weights.quantile(0.75)

# Combine return and volatility data
df_performance = pd.DataFrame({
    'expected_return': returns_list,
    'expected_volatility': volatility_list
})

# Calculate mean return and volatility
mean_return = df_performance['expected_return'].mean()
iqr_25_ret = df_performance['expected_return'].quantile(0.25)
iqr_75_ret = df_performance['expected_return'].quantile(0.75)

mean_volatility = df_performance['expected_volatility'].mean()
iqr_25_vol = df_performance['expected_volatility'].quantile(0.25)
iqr_75_vol = df_performance['expected_volatility'].quantile(0.75)

iqr_25_dict = {}
iqr_75_dict = {}
mean_dict = {}

for index, row in df_weights.iterrows():
    row_sum = row.sum()
    shortfall = 100 - row_sum
    null_columns = row[row.isnull()].index  # Identify columns with NaN values
    df_weights.loc[index, null_columns] = shortfall / len(null_columns)  # Distribute the shortfall evenly

    # Convert weights to a tuple for hashability
    # tuple_weights = tuple(row.tolist())

In [364]:
returns_df = pd.DataFrame(returns_list)
lower_threshold = [x for x in returns_df[0] if x < 2.00]

In [365]:
len(lower_threshold)

184

In [319]:
iqr_25_dict['weights'] = iqr_25_weights
iqr_75_dict['weights'] = iqr_75_weights
mean_dict['weights'] = mean_weights

iqr_25_dict['ret'] = iqr_25_ret
iqr_75_dict['ret'] = iqr_75_ret
mean_dict['ret'] = mean_return

iqr_25_dict['vol'] = iqr_25_vol
iqr_75_dict['vol'] = iqr_75_vol
mean_dict['vol'] = mean_volatility

In [320]:
weights = {}
all_dicts = {}

all_dicts['iqr25'] = iqr_25_dict
all_dicts['iqr75'] = iqr_75_dict
all_dicts['mean_dict'] = mean_dict

for df, items in all_dicts.items():
    highest_ret = 0
    if all_dicts[df]['ret'] > highest_ret:
        highest_ret = all_dicts[df]['ret']
        weights = all_dicts[df]['weights']

In [309]:
def scale_weights(weights):

    total_weights = __builtins__.sum(weights.values)
    scaled = {currency: weight / total_weights * 100 for currency, weight in weights.items()}

    if isinstance(weights, pd.Series):
        return pd.Series(scaled)
    return scaled

In [310]:
weights = scale_weights(weights)

### scrap code

In [393]:
expected_return_lst = {}
for df, values in stats.items():
    mean = values['mean']
    std = values['std']
    
    for df2, lr_model in models.items():
        if df == df2:
            print(f'match found for {df} and {df2}')
            # Regression Model calculating expected returns 
            # expected_return_lst = {}
            random_eer_ret = np.random.normal(mean, std, 1).reshape(-1,1)
            print(lr_model.predict(random_eer_ret))
            expected_return_lst[df2] = lr_model.predict(random_eer_ret)
            

match found for AU and AU
[0.00976244]
match found for CA and CA
[-0.01374206]
match found for CH and CH
[0.00070739]
match found for GB and GB
[0.00295269]
match found for JP and JP
[-0.00557417]
match found for NZ and NZ
[-0.0166149]
match found for XM and XM
[0.00791541]


In [509]:
lst = []
for i in expected_return_lst.values():
    lst.append(i)

In [500]:
highest_return = max(lst)
desired_scale = 0.05  # 5%
scaling_factor = desired_scale / highest_return

In [501]:
# Apply the scaling factor
scaled_returns = [ret * scaling_factor for ret in lst] 
scaled_returns 

[array([0.05]),
 array([-0.07038234]),
 array([0.00362304]),
 array([0.01512269]),
 array([-0.02854906]),
 array([-0.08509608]),
 array([0.04054015])]

In [569]:
scaled_values = [x[0] * 100 for x in scaled_returns]

In [578]:
fin = []
for i in scaled_values:
    fin.append(abs(i))

In [585]:
from pypfopt import EfficientFrontier, risk_models, expected_returns

# Assuming 'data' is your DataFrame with historical returns
# Calculate expected returns and the covariance matrix
mu = fin
S = risk_models.sample_cov(all_returns)

# Optimize for the minimised volatility
ef = EfficientFrontier(mu, S, weight_bounds = (0.05, 0.25))
weights = ef.efficient_risk(0.025, market_neutral = False)
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

# Performance metrics
expected_return, expected_volatility, expected_sharpe = ef.portfolio_performance()

OrderedDict([('AUDUSD.a_close', 0.05), ('USDCAD.a_close', 0.25), ('USDCHF.a_close', 0.12749), ('GBPUSD.a_close', 0.05), ('USDJPY.a_close', 0.14772), ('NZDUSD.a_close', 0.2432), ('EURUSD.a_close', 0.13159)])


In [588]:
da = discrete_allocation.DiscreteAllocation(
    cleaned_weights, latest_prices, 
    total_portfolio_value = 10000, short_ratio = None)

In [591]:
# Perform the discrete allocation
allocation, leftover = da.lp_portfolio()
# Calculate the total investment in each asset
total_investment = {ticker: allocation[ticker] * latest_prices[ticker] for ticker in allocation}

# Total value of the portfolio
total_portfolio_value = sum(total_investment.values()) + leftover

# Convert to percentages
portfolio_percentages = {ticker: (value / total_portfolio_value) * 100 for ticker, value in total_investment.items()}

In [ ]:
import numpy as np

num_simulations = 1000
all_weights = np.zeros((num_simulations, len(data.columns)))

for i in range(num_simulations):
    # Simulate the returns - this is a simple simulation; adjust as needed
    simulated_returns = np.random.normal(mu, S, size=(len(data), len(data.columns)))
    
    # Re-run MVO for the simulated returns
    ef = EfficientFrontier(simulated_returns.mean(axis=0), risk_models.sample_cov(pd.DataFrame(simulated_returns)))
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    
    # Store the weights from each simulation
    all_weights[i, :] = np.array(list(cleaned_weights.values()))

# Orders 
- Create a class that takes in ALL orders before sending through so you can run a monte carlo simulation of portfolio of total portfolio to get the expected std's etc. Adjust as needed with a final MVO. Target STD/Volatility for each week is 2.5% 
Monte carlo simulation of all orders going through

### Simple Order Send
Copying EER return direction as data is released.

In [25]:
symbols = []
for sym in count_codes:
    symbols.append(sym[0])

In [26]:
all_pairs = list(combinations(symbols, 2))

NameError: name 'combinations' is not defined

In [ ]:
def adf_test(series1, series2):
    '''Runs ADF test on a spread series'''
    spread = series1 - series2
    result = adfuller(spread)
    return {'ADF Statistic': result[0], 'p-value': result[1], 'Critical Values': result[4]}

for sym in all_pairs:
    series1 = get_rates(sym[0], mt5.TIMEFRAME_D1, 500

In [64]:
def sim_send():
    for symbol, value in last_update.items():
        if symbol[:3] == 'USD':
            print(f"Looping through {symbol}")
            if value > 0:
                send_order(symbol, 'sell', 1.00, 'EER')
                print(f"Selling {symbol}")
            else:
                send_order(symbol, 'buy', 1.00, 'EER')
        else:
            if value > 0:
                send_order(symbol, 'buy', 1.00, 'EER')
            else:
                send_order(symbol, 'sell', 1.00, 'EER')

In [875]:
def mc_weights_send():
    for symbol, value in last_update.items():
        print(f"Looping through {symbol}")

        # Find the corresponding weight for the symbol
        for sym, weight in weights.items():
            if symbol[:7] == sym[:7]:  # Assuming symbol format like 'AUDUSD.a'
                lotsize = weight
                break
        else:
            continue  # If no matching weight is found, continue to next symbol

        # Determine whether to buy or sell based on the value
        lotsize = round(( weight / 7 ), 2)
        
        if symbol[:3] == 'USD':
            print(f"Looping through {symbol}")
            if value > 0:
                send_order(symbol, 'sell', lotsize, 'MC_EER')
                print(f"Selling {symbol}")
            else:
                send_order(symbol, 'buy', lotsize, 'MC_EER')
        else:
            if value > 0:
                send_order(symbol, 'buy', lotsize, 'MC_EER')
            else:
                send_order(symbol, 'sell', lotsize, 'MC_EER')

In [876]:
mc_weights_send()

Looping through AUDUSD.a
Looping through USDCAD.a
Looping through USDCAD.a
Selling USDCAD.a
Looping through USDCHF.a
Looping through USDCHF.a
Looping through GBPUSD.a
Looping through USDJPY.a
Looping through USDJPY.a
Selling USDJPY.a
Looping through NZDUSD.a
Looping through EURUSD.a


### Pair / Triplet Trading

Trading based on a cointegrating relationship within EER values

### Mean Reversion / Momentum 

Based on mean-reverting nature of EER values balanced with the momentum of EER values

# Research

Analyzing the relationship of EER data and close prices. 

In [737]:
for df in weekly_dfs.values():
    df['ret'] = df['close'].pct_change()
    df['ret_diff'] = df['EER_ret'] - df['ret']
    df = df.dropna()

In [738]:
for name, df in weekly_dfs.items():
    print(f"EER Correlation with {name} is: {df['EER_ret'].corr(df['ret'])}")

EER Correlation with AU is: 0.7895590513032674
EER Correlation with CA is: -0.8034449420507715
EER Correlation with CH is: -0.6962195656612519
EER Correlation with GB is: 0.6871875814155363
EER Correlation with JP is: -0.8164550108813893
EER Correlation with NZ is: 0.774779037008523
EER Correlation with XM is: 0.7549860097389316


In [739]:
desc_stats = pd.DataFrame()

for name in weekly_dfs:
    desc_stats[f"{name} Stats"] = weekly_dfs[name]['ret_diff'].describe()
desc_stats

,AU Stats,CA Stats,CH Stats,GB Stats,JP Stats,NZ Stats,XM Stats
count,499.000000,499.000000,499.000000,499.000000,499.000000,499.000000,499.000000
mean,0.000299,-0.000728,0.000471,0.000336,-0.001096,0.000350,0.000288
std,0.009047,0.018505,0.020930,0.009593,0.022437,0.009860,0.007556
min,-0.053037,-0.064334,-0.104875,-0.050289,-0.100707,-0.055615,-0.033891
25%,-0.005061,-0.011256,-0.009507,-0.005547,-0.013601,-0.005001,-0.004506
50%,0.000292,-0.000322,-0.000457,0.000599,-0.002314,0.000192,0.000589
75%,0.005601,0.011485,0.009052,0.005842,0.011016,0.005819,0.004759
max,0.033373,0.059054,0.284139,0.036804,0.077664,0.039223,0.029442


In [740]:
eer_vals = pd.DataFrame()

for name in weekly_dfs:
    eer_vals[f'{name}'] = weekly_dfs[name]['OBS_Value']

In [741]:
from statsmodels.tsa.stattools import coint
from itertools import combinations
import pandas as pd
from statsmodels.tsa.stattools import adfuller 

def cointegration_test(series1, series2):
    '''Runs cointegration test on two series'''
    score, p_value, _ = coint(series1, series2)
    return {'Cointegration Score': score, 'p-value': p_value}

In [742]:
all_pairs = list(combinations(eer_vals.columns, 2))
all_trips = list(combinations(eer_vals.columns, 3))

In [743]:
def adf_test(series1, series2):
    '''Runs ADF test on a spread series'''
    spread = series1 - series2
    result = adfuller(spread)
    return {'ADF Statistic': result[0], 'p-value': result[1], 'Critical Values': result[4]}

def johansen_test(data, det_order=0, k_ar_diff=1):

    result = coint_johansen(data, det_order, k_ar_diff)
    return result


In [744]:
# Perform cointegration test on each pair
coint_results = {}
for pair in all_pairs:
    series1, series2 = eer_vals[pair[0]], eer_vals[pair[1]]
    coint_test_result = adf_test(series1, series2)
    if coint_test_result['Critical Values']['10%'] > coint_test_result['ADF Statistic']:
        # print(f"{coint_test_result['Critical Values']['10%']} is smaller than {coint_test_result['ADF Statistic']}. Adding to results")
        coint_results[pair] = {'ADF_Stat': round(coint_test_result['ADF Statistic'], 4), 
                               '10%_stat': round(coint_test_result['Critical Values']['10%'], 4)}
coint_results

{('AU', 'NZ'): {'ADF_Stat': -4.1478, '10%_stat': -2.5699},
 ('CA', 'NZ'): {'ADF_Stat': -4.1739, '10%_stat': -2.5699},
 ('CA', 'XM'): {'ADF_Stat': -2.6991, '10%_stat': -2.5699},
 ('NZ', 'XM'): {'ADF_Stat': -2.571, '10%_stat': -2.5699}}

In [745]:
eer_vals

,AU,CA,CH,GB,JP,NZ,XM
date,,,,,,,
2014-05-25,119.55,118.63,90.36,114.02,95.19,113.35,105.64
2014-06-01,120.61,119.35,90.36,113.41,95.38,112.25,105.57
2014-06-08,121.06,118.37,90.58,113.84,94.61,112.78,105.71
2014-06-15,121.81,119.26,90.29,115.36,95.10,114.50,104.76
2014-06-22,121.76,119.58,90.48,115.60,94.96,114.89,105.11
...,...,...,...,...,...,...,...
2023-11-19,101.76,100.03,111.49,102.15,73.96,99.72,100.55
2023-11-26,102.54,100.19,111.62,102.95,73.69,100.72,100.65
2023-12-03,103.02,101.12,112.62,103.62,74.34,102.05,99.92


In [108]:
for sym in all_trips:
    data = eer_vals[sym[0]], eer_vals[sym[1]], eer_vals[sym[2]]
test_result = johansen_test(pd.DataFrame(data).T)
n = len(data)
for i in range(n):
    print(f"  At 90%: {test_result.trace_stat_crit_vals[i, 0]}, Trace Stat = {test_result.trace_stat[i]}")
    print(f"  At 95%: {test_result.trace_stat_crit_vals[i, 1]}, Trace Stat = {test_result.trace_stat[i]}")
    print(f"  At 99%: {test_result.trace_stat_crit_vals[i, 2]}, Trace Stat = {test_result.trace_stat[i]}")

  At 90%: 27.0669, Trace Stat = 26.334654627092632
  At 95%: 29.7961, Trace Stat = 26.334654627092632
  At 99%: 35.4628, Trace Stat = 26.334654627092632
  At 90%: 13.4294, Trace Stat = 7.676091331107257
  At 95%: 15.4943, Trace Stat = 7.676091331107257
  At 99%: 19.9349, Trace Stat = 7.676091331107257
  At 90%: 2.7055, Trace Stat = 0.7471535183749242
  At 95%: 3.8415, Trace Stat = 0.7471535183749242
  At 99%: 6.6349, Trace Stat = 0.7471535183749242


In [115]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
trip_coint_res = {}
for sym in all_trips:
    data = eer_vals[sym[0]], eer_vals[sym[1]], eer_vals[sym[2]]
    test_result = johansen_test(pd.DataFrame(data).T)
    
    if test_result.trace_stat_crit_vals[1,0] < test_result.trace_stat[1]:
        print(f"Result: {test_result.trace_stat_crit_vals[1,0]} < {test_result.trace_stat[1]}")
        print(f"{sym} cointegrates on a trip. level")
    
    # result = johansen_test(data)

Result: 13.4294 < 13.937368787819146
('AU', 'CA', 'GB') cointegrates on a trip. level
Result: 13.4294 < 19.290192302491445
('AU', 'CA', 'NZ') cointegrates on a trip. level
Result: 13.4294 < 17.71064339528753
('AU', 'CA', 'XM') cointegrates on a trip. level
Result: 13.4294 < 13.440846845315779
('AU', 'GB', 'XM') cointegrates on a trip. level
Result: 13.4294 < 18.203896258830596
('AU', 'NZ', 'XM') cointegrates on a trip. level
Result: 13.4294 < 15.570874597943478
('CA', 'CH', 'JP') cointegrates on a trip. level
Result: 13.4294 < 16.647381123213624
('CA', 'GB', 'NZ') cointegrates on a trip. level
Result: 13.4294 < 16.29832999361456
('CA', 'GB', 'XM') cointegrates on a trip. level
Result: 13.4294 < 23.233955281631843
('CA', 'NZ', 'XM') cointegrates on a trip. level
Result: 13.4294 < 14.135697301903793
('GB', 'NZ', 'XM') cointegrates on a trip. level


In [119]:
trip_coint_res = {}
for sym in all_trips:
    data = pd.DataFrame({sym[0]: eer_vals[sym[0]], sym[1]: eer_vals[sym[1]], sym[2]: eer_vals[sym[2]]})
    test_result = johansen_test(data)
    
    
    cointegrating_relations = sum(test_result.trace_stat > test_result.trace_stat_crit_vals[:, 1])
    if cointegrating_relations >= 3:
        trip_coint_res[sym] = cointegrating_relations

    # print(f"{sym} shows evidence of {cointegrating_relations} cointegrating relationship(s).")

# Output results
for trip, relations in trip_coint_res.items():
    print(f"{trip} has {relations} cointegrating relationship(s).")

('AU', 'CA', 'NZ') has 3 cointegrating relationship(s).
('AU', 'CA', 'XM') has 3 cointegrating relationship(s).
('AU', 'NZ', 'XM') has 3 cointegrating relationship(s).
('CA', 'GB', 'NZ') has 3 cointegrating relationship(s).
('CA', 'GB', 'XM') has 3 cointegrating relationship(s).
('CA', 'NZ', 'XM') has 3 cointegrating relationship(s).


In [124]:
eer_vals[['AU', 'CA', 'NZ']].describe()

,AU,CA,NZ
count,501.000000,501.000000,501.000000
mean,106.988363,104.074631,104.420279
std,5.515604,4.818668,4.175869
min,89.110000,93.230000,95.260000
25%,103.110000,101.220000,101.550000
50%,106.460000,102.940000,103.740000
75%,110.230000,105.950000,106.510000
max,124.420000,121.550000,116.580000


## Risk Management Framework

Aim is to take into account associated statistics between the two and minimise volatility whilst maximising return. Appropriate measures can include:
- MVO or BL Portfolio
- Correlation analysis and appropriate hedging associated 